# Visualization

In [1]:
from brian2 import *
from brian2modelfitting import *

import numpy as np

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


KeyboardInterrupt: 

In [ ]:
dt = 0.01*ms
defaultclock.dt = dt

## Load the Data

In [ ]:
voltage1 = genfromtxt('voltage1nA.csv', delimiter=',')
voltage2 = genfromtxt('voltage2nA.csv', delimiter=',')

In [ ]:
inp_cur1 = genfromtxt('inp_cur1nA.csv', delimiter=',')
inp_cur2 = genfromtxt('inp_cur2nA.csv', delimiter=',')

In [ ]:
inp_current = np.array([inp_cur1, inp_cur2])
inp_current

In [ ]:
out_spikes1 = genfromtxt('out_spikes1nA.csv', delimiter=',')
out_spikes2 = genfromtxt('out_spikes2nA.csv', delimiter=',')
out_spikes = [out_spikes1, out_spikes2]
# out_spikes

In [ ]:
my_data = genfromtxt('inp_cur1nA.csv', delimiter=',')


## Define Model

In [ ]:
# res={'taut': 0.038281479669330376,
#  'R': 8084051733.384311,
#  'D': 0.0025722558443824674,
#  'a': 0.08427460573972566,
#  'tau': 0.01007468391773357,
#  'alpha': 1.4880312994647356}

In [ ]:
res={'taut': 0.03940264530424703,
 'tau': 0.019428036380643388,
 'alpha': 1.0694762113655902,
 'R': 13154191074.086367,
 'a': 0.09021720088860954,
 'D': 0.00047695430996046867}

In [ ]:

model = '''
        dv/dt = (R*I(t, i)- v)/tau : 1
        dvt/dt = (a*v - vt) / taut :1
        alpha : 1 (constant)
        a : 1 (constant)
        R : 1/amp (constant)
        tau : second (constant)
        taut : second (constant)
        D : second(constant)
        '''

reset = '''
v = 0
vt = vt + alpha
'''

In [ ]:
I = TimedArray(inp_current.transpose()*nA, dt=dt)

In [ ]:
start_scope()
G = NeuronGroup(2, model,
                threshold='v > 1 + vt',
                reset=reset,
                refractory='D',
                method='exponential_euler')

G.set_states(res, units=False)
mon = StateMonitor(G, ['v', 'vt'], record=True)
spike_mon = SpikeMonitor(G)

In [ ]:
run(50*second)

In [ ]:
fits = mon.v[:]
fits_vt = mon.vt[:]

In [ ]:
all_values = spike_mon.all_values()
spikes1 = all_values['t'][0]/ms
spikes2 = all_values['t'][1]/ms

In [ ]:
t0 = np.arange(0, len(fits[0]))*dt/ms

In [ ]:
np.savetxt('fits_vt_hh.csv', fits_vt, delimiter=",")
np.savetxt('fits_hh.csv', fits, delimiter=',')
np.savetxt('spikes1_hh.csv', spikes1, delimiter=',')
np.savetxt('spikes2_hh.csv', spikes2, delimiter=',')
